In [2]:
import torch
from datasets import load_dataset


# 定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self,split):
        dataset = load_dataset(path='seamew/ChnSentiCorp',split=split)

        def f(data):
            return len(data['text']) > 30  # 只要长度大于30的
        self.dataset = dataset.filter(f)
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,i):
        text = self.dataset[i]['text']
        return text

dataset = Dataset('train')
len(dataset),dataset[0]

Using custom data configuration default
Reusing dataset chn_senti_corp (/home/qilb/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/10 [00:00<?, ?ba/s]

(9192,
 '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般')

In [14]:
from transformers import BertTokenizer
# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [15]:
# 定义批处理函数
def collate_fn(data):


    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs = data,
                                    truncation = True,
                                    padding = 'max_length',
                                    max_length = 30,
                                    return_tensors = 'pt',
                                    return_length=True)
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']


    # 把原来input_ids 中第15个词作为label
    labels = input_ids[:,15].reshape(-1).clone()
    # 把input_ids中第15个词固定替换为mask 
    input_ids[:,15] = token.get_vocab()[token.mask_token]
    
    return input_ids,attention_mask,token_type_ids,labels

# 定义数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                    batch_size = 16,
                                    collate_fn = collate_fn,
                                    shuffle = True,
                                    drop_last = True)
for i,(input_ids,attention_mask,token_type_ids,labels) in enumerate(loader):
    break
print (len(loader))
print (token.decode(input_ids[0]))
print (token.decode(labels[0]))
input_ids.shape,attention_mask.shape,token_type_ids.shape,labels

574
[CLS] 质 量 不 错 ， 适 合 3 岁 左 右 小 孩 玩 [MASK] 每 一 页 边 上 都 有 已 经 打 好 的 虚 [SEP]
。


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 tensor([ 511, 3221,  510, 6858, 4905, 1914, 3221, 1057,  677, 6230,  673,  678,
         2769, 1184, 3315, 8024]))

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from transformers import BertModel
#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:

# 定义下游任务模型  本质是在分类  看待填空的那个字是词典中的哪一个
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768,token.vocab_size,bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias
    def forward(self,input_ids,attention_mask,token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            print (out.last_hidden_state[:,15].shape)
        out = self.decoder(out.last_hidden_state[:,15]) # 这里固定是第15个词 实际应用可能是任何一个位置的 需记录索引
        return out
model = Model()
model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids).shape


torch.Size([16, 768])


torch.Size([16, 21128])

In [29]:

#训练模型
from transformers import AdamW
# 训练
optimizer = AdamW(model.parameters(),lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()
model.train()
for i, (input_ids,attention_mask,token_type_ids,labels) in enumerate(loader):
    out  = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
    loss = criterion(out,labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i%50  == 0:
        out = out.argmax(dim = 1)
        accuracy = (out == labels).sum().item() / len(labels)
        print (i, loss.item(),accuracy)
    # if i == 3000:
    #     break


0 2.2244198322296143 0.5625
50 1.3433104753494263 0.6875
100 1.7187391519546509 0.6875
150 1.9837058782577515 0.6875
200 1.6465390920639038 0.625
250 1.3356690406799316 0.875
300 0.9679287672042847 0.875
350 1.6963425874710083 0.625
400 0.35137617588043213 0.9375
450 1.0924913883209229 0.8125
500 1.4483717679977417 0.8125
550 0.7847290635108948 0.875


In [35]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),batch_size=32,collate_fn=collate_fn,shuffle=True,drop_last=True)
    for i, (input_ids,attention_mask,token_type_ids,labels) in enumerate(loader_test):
        if i == 16:
            break
        print (i)
        with torch.no_grad():

            out  = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)

    
        out = out.argmax(dim = 1)
        correct += (out == labels).sum().item()
        total += len(labels)
        print (token.decode(input_ids[0]))
        print (token.decode(labels[0]),token.decode(labels[0]))
    print (correct / total)
test()

Using custom data configuration default
Reusing dataset chn_senti_corp (/home/qilb/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/2 [00:00<?, ?ba/s]

0
[CLS] 说 实 话 ， 写 的 实 在 不 怎 么 样 ， 东 [MASK] 西 凑 ， 主 人 公 那 样 的 心 理 素 质 [SEP]
拼 拼
1
[CLS] 1 、 有 点 重 ， 2 、 速 度 明 显 较 14 [MASK] 的 笔 记 本 慢 ， 打 开 网 页 多 的 时 [SEP]
寸 寸
2
[CLS] 实 在 太 远 了 ， 到 香 港 就 是 满 足 购 [MASK] 需 要 ， 留 在 酒 店 出 门 不 方 便 。 [SEP]
物 物
3
[CLS] 键 盘 太 拥 挤 按 着 不 太 舒 服, 也 容 [MASK] 按 错 键. 不 过 这 体 积 大 概 也 只 [SEP]
易 易
4
[CLS] 原 本 在 网 上 订 了 两 个 套 房 ， 入 住 [MASK] ， 携 程 还 给 我 打 电 话 问 是 否 只 [SEP]
后 后
5
[CLS] 配 置 算 不 错 ， 装 [UNK] 有 点 麻 烦 ， 关 [MASK] 是 找 有 [UNK] 驱 动 的 盘 ， 按 照 网 上 [SEP]
键 键
6
[CLS] 首 先 附 赠 软 件 （ [UNK] 、 [UNK] ） 是 在 第 [MASK] 次 启 动 时 决 定 是 否 安 装 。 不 像 [SEP]
一 一
7
[CLS] 还 不 如 直 接 到 那 里 找 个 导 游 去 订 [MASK] 比 携 程 订 的 便 宜, 导 游 说 十 一 [SEP]
都 都
8
[CLS] 外 观 设 计 别 出 心 裁 ！ 配 置 均 衡 性 [MASK] 比 高 ， 比 [UNK] 系 列 又 有 进 步 。 散 [SEP]
价 价
9
[CLS] 这 次 入 住 少 林 宾 馆 还 是 比 较 满 意 [MASK], 宾 馆 的 位 置 比 较 好 找, 从 客 [SEP]
的 的
10
[CLS] 屏 的 显 示 效 果 好 象 不 大 好 ， 不 知 [MASK] 是 才 拿 到 不 会 调 还 是 其 它 问 题 [SEP]
道 道
11
[CLS] 很 一 般 的 书 ， 女 儿 不 是 很 喜 欢 ， [MASK] 介 绍 买 的 ， 以 为 应 该 不 错 的 ， [SEP]
看 看
12
[CLS] 我 十 一 来 住 了 两 天 ， 一 进 门 就